In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.utils import image_dataset_from_directory
from keras import layers, Model
from PIL import Image
from tensorflow import clip_by_value
from tensorflow import data as tf_data
from tensorflow import image as tf_image
from tensorflow import random as tf_random
import keras_cv

In [ ]:
train_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='training',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    # label_mode = 'categorical'
    )

class_names = train_ds.class_names

val_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='validation',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    # label_mode = 'categorical'
    )

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_ds.class_names[labels[i]])
    plt.axis("off")
plt.savefig('../media/cinic_10_examples.jpg')

# below code is for data augmentations, applied during training, on batches, as layers of the model.

there's several layers:
- RescaleLayer
- RotateLayer
- FlipLayer
- DecolorizeLayer
- GaussianNoiseLayer

we use those at the beginning of our model

In [ ]:
# rescale to 0-1 range
RescaleLayer = layers.Rescaling(1./255)

### rotations

In [ ]:
RotateLayer = layers.RandomRotation(0.2)

### Flipping

In [ ]:
FlipLayer = layers.RandomFlip("horizontal")

### Decolorization

In [ ]:
DecolorLayer = keras_cv.layers.Grayscale(output_channels=3)

### noise

In [ ]:
GaussianNoise = layers.GaussianNoise(stddev = 0.05)

In [ ]:
def get_var_name(var):
    for name, value in locals().items():
        if value is var:
            return name
        
image = images[0]
image = RescaleLayer(image)
fig, axes = plt.subplots(2,3, figsize=(12, 12))
axes = axes.flatten()
axes[0].imshow(image)
axes[0].set_title('original image')
axes[0].axis('off')
for augmentationLayer, ax in zip([RotateLayer, FlipLayer, DecolorLayer, GaussianNoise], axes[1:]):
    ax.imshow(augmentationLayer(image, training = True), cmap='grey' if augmentationLayer == DecolorLayer else None)
    ax.set_title('Rescaled (visually does nothing)' if augmentationLayer == RescaleLayer else 
                 'Rotated' if augmentationLayer == RotateLayer else 
                 'Flipped' if augmentationLayer == FlipLayer else 
                 'Decolored' if augmentationLayer == DecolorLayer else 
                 'Noisy')
    ax.axis('off')
axes[-1].remove()
plt.savefig('../media/standard_augmentations_example.jpg', dpi = 200)

In [ ]:
IMAGE_SIZE = 32
num_classes = 10
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
def to_dict(image, label):
    # image = tf.image.resize(image, IMAGE_SIZE)
    # image = tf.cast(image, tf.float32)
    # label = tf.one_hot(label, num_classes)
    return {"images": image, "labels": label}


def prepare_dataset(dataset, split):
    if split == "train":
        return (
            dataset.shuffle(10 * BATCH_SIZE)
            .map(to_dict, num_parallel_calls=AUTOTUNE)
            .batch(BATCH_SIZE)
        )
    if split == "test":
        return dataset.map(to_dict, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE)

In [ ]:
cut_mix = keras_cv.layers.CutMix()

def cutmix(samples):
    samples = cut_mix(samples, training=True)
    return samples
# dict_train_ds = train_ds.map(to_dict, num_parallel_calls=AUTOTUNE)
# dict_train_ds = dict_train_ds.map(cutmix, num_parallel_calls=AUTOTUNE)
def dict_to_tuple(element):
    return element['images'], element['labels']

# cutmixed_ds = dict_train_ds.map(dict_to_tuple)

In [ ]:
train_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='training',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical'
    )

class_names = train_ds.class_names

val_ds = image_dataset_from_directory(
    '../data/cinic-10_image_classification_challenge-dataset/train/',
    validation_split=0.2,
    subset='validation',
    seed = 420,
    image_size=(32,32),
    batch_size=32,
    label_mode = 'categorical'
    )

In [ ]:
dict_train_ds = train_ds.take(1).map(to_dict, num_parallel_calls=AUTOTUNE)
dict_train_ds = dict_train_ds.map(cutmix, num_parallel_calls=AUTOTUNE)
cutmixed_ds = dict_train_ds.map(dict_to_tuple)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in cutmixed_ds:
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i+9].numpy().astype("uint8"))
    label = ""
    for j, val in enumerate(labels[i+9]):
            if val > 0:
                label += class_names[j] + str(np.round(val, 2)) + " "
    plt.title([label])
    plt.axis("off")
plt.savefig('../media/cutmix_example.jpg', dpi = 200)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from keras import datasets, layers, models, losses, Model, utils
import time
import numpy as np

# rotation

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
inputs = layers.Input((32,32,3))
x = RotateLayer(inputs)

x = base_model50(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

# Pass the input through the Random Rotation layer
inputs = layers.Input(shape=(32, 32, 3))
x = RotateLayer(inputs)

# Continue with the existing model architecture
x = base_model101(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# Flipping

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
inputs = layers.Input((32,32,3))
x = FlipLayer(inputs)

x = base_model50(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

# Pass the input through the Random Rotation layer
inputs = layers.Input(shape=(32, 32, 3))
x = FlipLayer(inputs)

# Continue with the existing model architecture
x = base_model101(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# Decolorization

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
inputs = layers.Input((32,32,3))
x = DecolorLayer(inputs)

x = base_model50(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

# Pass the input through the Random Rotation layer
inputs = layers.Input(shape=(32, 32, 3))
x = DecolorLayer(inputs)

# Continue with the existing model architecture
x = base_model101(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# Noise

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
inputs = layers.Input((32,32,3))
inputs = RescaleLayer(inputs)
x = GaussianNoise(inputs)

x = base_model50(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

# Pass the input through the Random Rotation layer
inputs = layers.Input(shape=(32, 32, 3))
inputs = RescaleLayer(inputs)
x = GaussianNoise(inputs)

# Continue with the existing model architecture
x = base_model101(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# CUTMIX

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
  
input_images = layers.Input((32, 32, 3), name = 'images')
input_images = layers.Rescaling(1./255)(input_images)
input_labels = layers.Input((10,), name = 'labels')
# Apply CutMix to images here
inputs = keras_cv.layers.CutMix()({'images' : input_images, 'labels' : input_labels})['images']
x = base_model50(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

# Pass the input through the Random Rotation layer
input_images = layers.Input((32, 32, 3), name = 'images')
input_labels = layers.Input((10,), name = 'labels')
# Apply CutMix to images here
inputs = keras_cv.layers.CutMix()({'images' : input_images, 'labels' : input_labels})['images']

# Continue with the existing model architecture
x = base_model101(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# combined augmentations

In [ ]:
pipeline = keras_cv.layers.RandomAugmentationPipeline(
    layers=[keras_cv.layers.Grayscale(output_channels=3),
            layers.RandomRotation(0.2),
            layers.RandomFlip("horizontal"),
            layers.GaussianNoise(stddev = 0.1)
            ],
    augmentations_per_image=2,
)

def apply_pipeline(images):
    images = pipeline(images)
    return images

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = False
  
input_images = layers.Input((32, 32, 3), name = 'images')
input_labels = layers.Input((10,), name = 'labels')
# Apply CutMix to images here
inputs = keras_cv.layers.CutMix()({'images' : input_images, 'labels' : input_labels})['images']
inputs = layers.Lambda(apply_pipeline, output_shape=(32, 32, 3))(inputs) # rest of the augmentations here
x = base_model50(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=5)

In [ ]:
# Load ResNet101 base model
base_model101 = tf.keras.applications.ResNet101(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze base model layers
for layer in base_model101.layers:
    layer.trainable = False

input_images = layers.Input((32, 32, 3), name = 'images')
input_labels = layers.Input((10,), name = 'labels')
# Apply CutMix to images here
inputs = keras_cv.layers.CutMix()({'images' : input_images, 'labels' : input_labels})['images']
inputs = layers.Lambda(apply_pipeline, output_shape=(32, 32, 3))(inputs) # rest of the augmentations here


x = base_model101(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.1))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model101 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model101.compile(optimizer="Adagrad", loss="categorical_crossentropy", metrics=["accuracy"])

# Train model
history101 = head_model101.fit(train_ds, validation_data=val_ds, epochs=5)

# long training

In [ ]:
pipeline = keras_cv.layers.RandomAugmentationPipeline(
    layers=[keras_cv.layers.Grayscale(output_channels=3),
            layers.RandomRotation(0.2),
            layers.RandomFlip("horizontal"),
            layers.GaussianNoise(stddev = 0.05)
            ],
    augmentations_per_image=2,
)

def apply_pipeline(images):
    images = pipeline(images)
    return images

In [ ]:
base_model50 = tf.keras.applications.ResNet50(weights = 'imagenet', include_top = False, input_shape = (32,32,3))
for layer in base_model50.layers:
  layer.trainable = True
  
input_images = layers.Input((32, 32, 3), name = 'images')
input_labels = layers.Input((10,), name = 'labels')
# Apply CutMix to images here
inputs = keras_cv.layers.CutMix()({'images' : input_images, 'labels' : input_labels})['images']
inputs = layers.Lambda(apply_pipeline, output_shape=(32, 32, 3))(inputs) # rest of the augmentations here
x = base_model50(inputs, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1000, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.05))(x)
predictions = layers.Dense(10, activation='softmax')(x)

# Create model
head_model50 = Model(inputs=inputs, outputs=predictions)

# Compile model
head_model50.compile(optimizer="Adagrad", loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1), metrics=["accuracy"])

# Train model
history50 = head_model50.fit(train_ds, validation_data = val_ds, epochs=25)
head_model50.save_weights("head_model50_trained_whole.weights.h5")
base_model50.save_weights("base_model50_trained_whole.weights.h5")